In [2]:
path_folder = "../projected_by_linear_model_to_2050"
PET = path_folder+"/petprojected_amounts_to_relog_grouped_2050.csv"
PAPER = path_folder+"/paperprojected_amounts_to_relog_grouped_2050.csv"
OTHER = path_folder+"/otherprojected_amounts_to_relog_grouped_2050.csv"
IRON = path_folder+"/ironprojected_amounts_to_relog_grouped_2050.csv"
HDPE = path_folder+"/hdpeprojected_amounts_to_relog_grouped_2050.csv"
GLASS = path_folder+"/glassprojected_amounts_to_relog_grouped_2050.csv"
FILM = path_folder+"/filmprojected_amounts_to_relog_grouped_2050.csv"
CARD = path_folder+"/cardboardprojected_amounts_to_relog_grouped_2050.csv"
ALM = path_folder+"/aluminumprojected_amounts_to_relog_grouped_2050.csv"
ALL_WASTE_FILES = [PET, PAPER, OTHER, IRON, HDPE, GLASS, FILM, CARD, ALM]

In [3]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

# we have to normalize the county names to match the URI data because 
# Miguel's scripts sucks
def normalize_county_name(name):
    # convert to lowercase
    name = name.lower()
    # replace "saint" with "st"
    name = re.sub(r'\bsaint\b', 'st', name)
    # remove periods after "st"
    name = re.sub(r'\bst\.', 'st', name)
    # remove other punctuation and extra spaces
    name = re.sub(r'[^\w\s]', '', name)
    name = re.sub(r'\s+', ' ', name).strip()
    # replace Sainte with Ste
    name = re.sub(r'\bsainte\b', 'ste', name)
    # replace De Soto with Desoto
    name = re.sub(r'\bde soto\b', 'desoto', name)
    return name

def combine_waste_data(file_paths, uri_file_path):
    assert Path(uri_file_path).exists(),uri_file_path.resolve()
    uri_df = pd.read_csv(uri_file_path)
    uri_df['NormalizedCounty'] = uri_df['toponymName'].apply(normalize_county_name)
    
    all_data = []
    waste_types = ['PET', 'Paper', 'Other', 'Iron', 'HDPE', 'Glass', 'Film', 'Card', 'ALM']
    
    for file_path, waste_type in zip(file_paths, waste_types):
        df = pd.read_csv(file_path)
        
        # Remove the 'Unnamed: 0' and 'index' columns if these exist
        if 'Unnamed: 0' in df.columns:
            df = df.drop('Unnamed: 0', axis="columns")
        if 'index' in df.columns:
            df = df.drop('index', axis="columns")
        
        df.columns = df.columns.str.replace('.0', '')
        year_columns = df.columns[df.columns.str.isnumeric()]
        
        # Melt the dataframe to convert years to a single column
        df_melted = df.melt(id_vars=['name', 'State', 'State_County', 'latitude (deg)', 'longitude (deg)'],
                            value_vars=year_columns,
                            var_name='Year', value_name=waste_type)
        
        # Convert Year to integer
        df_melted['Year'] = df_melted['Year'].astype(int)
        
        all_data.append(df_melted)
    
    # merge all dataframes
    combined_df = all_data[0]
    for df in all_data[1:]:
        combined_df = pd.merge(combined_df, df, on=['name', 'State', 'State_County', 'latitude (deg)', 'longitude (deg)', 'Year'])
    
    # self explanatory
    combined_df = combined_df.rename(columns={'name': 'County'})
    combined_df['NormalizedCounty'] = combined_df['County'].apply(normalize_county_name)
    
    # fun to find matching URI
    def find_uri(row, uri_df):
        state_matches = uri_df[uri_df['adminName1'] == row['State']]
        if not state_matches.empty:
            county_match = state_matches[state_matches['NormalizedCounty'].str.contains(row['NormalizedCounty'], case=False, na=False)]
            if not county_match.empty:
                return county_match.iloc[0]['uri']
        return np.nan

    # Apply the function to each row
    combined_df['uri'] = combined_df.apply(lambda row: find_uri(row, uri_df), axis=1)
    
    # Remove the temporary NormalizedCounty column
    combined_df = combined_df.drop('NormalizedCounty', axis=1)
    
    # Reorder columns
    columns_order = ['County', 'State', 'State_County', 'latitude (deg)', 'longitude (deg)', 'Year', 'uri'] + waste_types
    combined_df = combined_df[columns_order]
    
    return combined_df

ALL_WASTE_FILES = [PET, PAPER, OTHER, IRON, HDPE, GLASS, FILM, CARD, ALM]
URI_FILE = 'counties_uris.csv'
combined_waste_df = combine_waste_data(ALL_WASTE_FILES, URI_FILE)

print(combined_waste_df.head())

# missing_uris = combined_waste_df[combined_waste_df['uri'].isna()][['County', 'State', 'Year']].drop_duplicates()
# print("\nCounties with missing URIs:")
# print(missing_uris)

# missing_by_state = missing_uris.groupby('State').size().sort_values(ascending=False)
# print("\nNumber of counties with missing URIs by state:")
# print(missing_by_state)

combined_waste_df.to_csv('combined_waste_data_with_uri.csv', index=False)

# missing_uris.to_csv('missing_uris.csv', index=False)

      County           State              State_County  latitude (deg)  \
0  Abbeville  South Carolina  South Carolina_Abbeville       34.213809   
1     Acadia       Louisiana          Louisiana_Acadia       30.291497   
2   Accomack        Virginia         Virginia_Accomack       37.765944   
3        Ada           Idaho                 Idaho_Ada       43.447861   
4      Adair            Iowa                Iowa_Adair       41.328528   

   longitude (deg)  Year                               uri          PET  \
0       -82.460460  2004  https://www.geonames.org/4568959   177.339712   
1       -92.411037  2004  https://www.geonames.org/4314344   485.163465   
2       -75.757807  2004  https://www.geonames.org/4743865   272.786855   
3      -116.244456  2004  https://www.geonames.org/5583739  2564.278612   
4       -94.478164  2004  https://www.geonames.org/4846344    58.150904   

         Paper      Other        Iron         HDPE        Glass        Film  \
0   102.422894   6.448717